In [1]:

import os
import argparse
import yaml
import graphviz
from cwltool.main import main
from io import StringIO


## Stage the Landsat-9 scene using CWL

In [4]:
# create the YAML parameter file
with open("stage-in-params.yaml", "w") as f:
    yaml.dump({"reference": "https://planetarycomputer.microsoft.com/api/stac/v1/collections/landsat-c2-l2/items/LC09_L2SP_042033_20231015_02_T1"}, f)

In [ ]:
parsed_args = argparse.Namespace(
    podman=True,
    debug=False,
    outdir="./runs",
    workflow="cwl-workflow/stage-in.cwl",
    job_order=["stage-in-params.yaml"],
)

stream_out = StringIO()
stream_err = StringIO()

res = main(
    args=parsed_args,
    stdout=stream_out,
)

assert res == 0

## Invoke the application package with the staged Landsat-9 scene

In [ ]:

# create the YAML parameter file
with open("params.yaml", "w") as f:
    f.write(yaml.dump({"item": {"class": "Directory", "path": "runs/staged"}, "aoi": "-118.985,38.432,-118.183,38.938", "epsg": "EPSG:4326", "band": ["green", "nir08"]}))


In [ ]:


parsed_args = argparse.Namespace(
    debug=False,
    outdir="./runs",
    workflow="cwl-workflow/app-water-bodies.cwl",
    job_order=["params.yaml"],
)

stream_out = StringIO()
stream_err = StringIO()

res = main(
    args=parsed_args,
    stdout=stream_out,
)

assert res == 0

In [ ]:


parsed_args = argparse.Namespace(print_dot=True,workflow="cwl-workflow/app-water-bodies.cwl#main")

stream_out = StringIO()
stream_err = StringIO()

res = main(
    args=parsed_args,
    stdout=stream_out,
)
assert res == 0
graphviz.Source(stream_out.getvalue())

In [ ]:
os.getcwd()